In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from ntpath import join
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from gensim.parsing.preprocessing import remove_stopwords
import string


In [2]:
with open("sample.txt") as file_in:
    lines = []
    for line in file_in:
        lines.append(line)
doc_test=[]
for i in range(len(lines)):
    doc_test.append(lines[i].split('.'))

final_doc=[]
for i in range(len(doc_test)):
    for j in range(len(doc_test[i])):
        final_doc.append(doc_test[i][j])

without_stopwords=[]
for i in final_doc:
    filtered_sentence=remove_stopwords(i)
    without_stopwords.append(filtered_sentence)

without_SandP = []
for i in without_stopwords:
    filtered_sentence = i.translate(str.maketrans('', '', string.punctuation))
    without_SandP.append(filtered_sentence)

without_SandP


['The World Soil Day observed today globally Nepal objective raising public awareness significance healthy soil sustainable management soil fertility',
 'The United Nations General Assembly December 2013 declared December 5 2014 World Soil Day formally marked world',
 'Nepal observing Day 2015',
 'The theme World Soil Day year Soils food begins',
 ' Soil heart agricultural activities food security nutrition security climate conservation',
 'The World Soil Day programme reiterates importance soil mankind crucial need conservation proper management time increasing fertility Department Agriculture said',
 'Director General Department Dr Rewati Raman Poudel said debate food nutritional security sustainable agriculture development conservation biodiversity organic agriculture meaning conservation promotion proper management soil',
 'The soil fertility deteriorating world including Nepal recent years declining physical chemical biological features soil',
 'Therefore problem declining soil fe

In [3]:
# Converting each document into an vector
vectorizer = CountVectorizer()

bag_of_words = vectorizer.fit_transform(without_SandP)


In [4]:
# print(bag_of_words)

bag_of_words.todense()

# print(bag_of_words.todense())


matrix([[0, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [5]:
# Singular value decomposition
# This process encodes our original data into topic encoded data
svd = TruncatedSVD(n_components=1)
lsa = svd.fit_transform(bag_of_words)


In [6]:
# Using pandas to look at the output of lsa

topic_encoded_df = pd.DataFrame(lsa, columns=["topic1"])
topic_encoded_df["without_stopwords"] = without_stopwords
topic_encoded_df[["without_stopwords", "topic1"]]


,without_stopwords,topic1
0,The World Soil Day observed today globally Nep...,3.714733
1,The United Nations General Assembly December 2...,2.004657
2,Nepal observing Day 2015,0.342729
3,The theme World Soil Day year 'Soils: food begins,1.723304
4,"' Soil heart agricultural activities, food sec...",1.341709
5,The World Soil Day programme reiterates import...,3.037662
6,"Director General Department, Dr Rewati Raman P...",2.169059
7,The soil fertility deteriorating world includi...,2.241480
8,"Therefore, problem declining soil fertility ta...",0.954158
9,Poudel said World Soil Day marked main goal ra...,2.657310


In [7]:
dictionary = vectorizer.get_feature_names()
print(dictionary)


['2013', '2014', '2015', '2022', '70', 'activities', 'addressing', 'aegis', 'agricultural', 'agriculture', 'aims', 'amidst', 'animals', 'assembly', 'awareness', 'begins', 'billion', 'biodiversity', 'biological', 'campaign', 'capacity', 'cause', 'celebrated', 'central', 'centre', 'challenges', 'change', 'chemical', 'climate', 'common', 'concentration', 'conservation', 'crops', 'crucial', 'day', 'debate', 'december', 'declared', 'declining', 'decreased', 'degradation', 'department', 'depleted', 'detect', 'deteriorating', 'development', 'difficult', 'director', 'dr', 'drastically', 'economic', 'ecosystems', 'encouraging', 'enterprises', 'environment', 'estimated', 'extensive', 'features', 'fertility', 'food', 'formally', 'general', 'global', 'globally', 'goal', 'growing', 'health', 'healthy', 'heart', 'hidden', 'high', 'human', 'hunger', 'importance', 'improve', 'improvement', 'including', 'increasing', 'induces', 'known', 'laboratory', 'lack', 'level', 'losing', 'loss', 'main', 'maintain

c:\Users\Kshitiz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
encoding_matrix = pd.DataFrame(svd.components_,index=["topic1"] ,columns=dictionary).T
encoding_matrix
# numerical values can be thought of as an expression of that word in respective topic



,topic1
2013,0.018845
2014,0.018845
2015,0.003222
2022,0.047172
70,0.006205
...,...
where,0.047172
world,0.262178
worldwide,0.006205
year,0.016200


In [9]:
#Considering the words affecting variance in data
encoding_matrix['abs_topic1']=np.abs(encoding_matrix)
encoding_matrix.sort_values('abs_topic1',ascending=False)

,topic1,abs_topic1
soil,0.683902,0.683902
world,0.262178,0.262178
day,0.225484,0.225484
food,0.205756,0.205756
the,0.177385,0.177385
...,...,...
estimated,0.006205,0.006205
detect,0.006205,0.006205
lack,0.006205,0.006205
observing,0.003222,0.003222


In [10]:
final_matrix=encoding_matrix.sort_values('abs_topic1',ascending=False)
final_matrix[["abs_topic1"]]

,abs_topic1
soil,0.683902
world,0.262178
day,0.225484
food,0.205756
the,0.177385
...,...
estimated,0.006205
detect,0.006205
lack,0.006205
observing,0.003222


In [30]:
# # Extracting out final sentence from topic 1
# sentence1 = final_matrix[final_matrix["abs_topic1"] >= 0.2]
# sentence1[['abs_topic1']]


,abs_topic1
soil,0.683902
world,0.262178
day,0.225484
food,0.205756


In [33]:
# Extracting out final sentence from topic 1
sentence1 = final_matrix.head(4)
sentence1[['abs_topic1']]

,abs_topic1
soil,0.683902
world,0.262178
day,0.225484
food,0.205756


In [12]:
index_list = list(sentence1.index.values)
index_list

['soil', 'world', 'day', 'food']

In [50]:
compression='large'
words_after_compression=[]
if compression=='small':
    words_after_compression.append(index_list[0])
elif compression == 'medium':
    words_after_compression.append(index_list[0])
    words_after_compression.append(index_list[1])
else :
    words_after_compression.extend(index_list)


words_after_compression

['soil', 'world', 'day', 'food']

In [51]:
final_conclusion=[]
for i in range(len(final_doc)):
    for j in range(len(words_after_compression)):
        if words_after_compression[j] in final_doc[i]:
            final_conclusion.append(final_doc[i])
    
final_conclusion

['The World Soil Day is being observed today globally as well as in Nepal with the objective of raising public awareness on the significance of healthy soil and for the sustainable management of the soil fertility',
 'The World Soil Day is being observed today globally as well as in Nepal with the objective of raising public awareness on the significance of healthy soil and for the sustainable management of the soil fertility',
 'The United Nations General Assembly had in December 2013 declared December 5, 2014 as the World Soil Day and it was formally marked throughout the world since then',
 " The theme of the World Soil Day this year is 'Soils: where food begins",
 "' Soil is at the heart of all agricultural activities, food security, nutrition security and climate conservation",
 ' The World Soil Day programme reiterates the importance of soil for mankind and the crucial need for its conservation and proper management while at the same time increasing its fertility, the Department 

In [46]:
list_final=list(set(final_conclusion))
# list_final

In [47]:
with open('final_output.txt', 'w') as filehandle:
    for listitem in list_final:
        filehandle.write('%s\n' % listitem)


In [48]:
possible_headings=[]
for i in index_list:
    for j in index_list:
        if i != j:
                possible_headings.append(i+' '+j)

# possible_headings

In [49]:
final_headings=[]
for i in possible_headings:
    split_list=i.split()
    sorted_list=sorted(split_list)
    final_headings.append(sorted_list)

final_headings_str=[]
for i in final_headings:
        final_headings_str.append(i[0]+' '+i[1])

ff_heading=list(set(final_headings_str))
ff_heading


['day soil', 'day food', 'soil world', 'food soil', 'day world', 'food world']

In [18]:
vectorizer = CountVectorizer()
bag_of_words1 = vectorizer.fit_transform(ff_heading)


In [19]:
bag_of_words1.todense()


matrix([[1, 0, 1, 0],
        [1, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [0, 1, 0, 1]], dtype=int64)

In [20]:
svd1 = TruncatedSVD(n_components=2)
lsa1 = svd1.fit_transform(bag_of_words1)


In [21]:
topic_encoded_df = pd.DataFrame(lsa1, columns=["topic1","topic2"])
topic_encoded_df["ff_heading"]= ff_heading
display(topic_encoded_df[["ff_heading","topic1","topic2"]])

,ff_heading,topic1,topic2
0,day soil,1.0,-0.57735
1,day food,1.0,-0.57735
2,soil world,1.0,0.57735
3,food soil,1.0,0.57735
4,day world,1.0,-0.57735
5,food world,1.0,0.57735


In [22]:
dictionary1 = vectorizer.get_feature_names()
dictionary1


c:\Users\Kshitiz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['day', 'food', 'soil', 'world']

In [23]:
encoding_matrix1 = pd.DataFrame(
    svd1.components_, index=["topic1", "topic2"], columns=dictionary1).T
encoding_matrix1


,topic1,topic2
day,0.5,-0.866025
food,0.5,0.288675
soil,0.5,0.288675
world,0.5,0.288675


In [24]:
encoding_matrix1['abs_topic1']=np.abs(encoding_matrix1["topic1"])
encoding_matrix1['abs_topic2']=np.abs(encoding_matrix1["topic2"])
encoding_matrix1.sort_values('abs_topic1',ascending=False)

,topic1,topic2,abs_topic1,abs_topic2
day,0.5,-0.866025,0.5,0.866025
food,0.5,0.288675,0.5,0.288675
soil,0.5,0.288675,0.5,0.288675
world,0.5,0.288675,0.5,0.288675


In [25]:
final_matrix1=encoding_matrix1.sort_values('abs_topic1',ascending=False)
final_matrix1[["abs_topic1","abs_topic2"]]

,abs_topic1,abs_topic2
day,0.5,0.866025
food,0.5,0.288675
soil,0.5,0.288675
world,0.5,0.288675


In [26]:

sentence3 = final_matrix1[final_matrix1["abs_topic2"] >= 0.4]
sentence3[['abs_topic2']]


,abs_topic2
day,0.866025


In [27]:
#selecting two words according to their absolute values.
count=0
heading_final_line=[]
heading_topic_final_df=[]
corr_sorted_desc = []
corr_values=[]
for i in final_matrix1["abs_topic2"]:
    corr_values.append(i)
corr_sorted_desc.extend(sorted(corr_values,reverse=True))
for i in corr_sorted_desc:
    if count==2:
        break
    heading_topic_final_df.append(final_matrix1[final_matrix1["abs_topic2"] == i])
    count=count+1


for i in heading_topic_final_df:
    heading_final_line.extend(i.index.values)

heading_final_line



['day', 'food']

In [28]:
#output 2-word heading in new file
with open('final_heading.txt', 'w') as filehandle:
    for listitem in heading_final_line:
        filehandle.write('%s\n' % listitem)
